In [ ]:
%pip install -r requirements.txt

In [ ]:
# scrape https://www.poetryfoundation.org/poems/browse#page=1&sort_by=recently_added for 40,000 poems?

In [109]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging
import re

In [124]:
df1 = pd.read_pickle('more_poems.pickle')

In [125]:
tags = [[] for _ in range(131)]

In [127]:
df1['tags'] = tags

In [128]:
df = pd.read_pickle('mostly_english_poems.pickle')
print(len(df))

16333


In [129]:
frames = [df, df1]

In [130]:
df = pd.concat(frames)

In [132]:
df.reset_index(drop=True, inplace=True)

In [134]:
df.to_pickle('mostly_english_poems.pickle')

In [92]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [93]:
Poems = df.content.map(lambda x: nltk.tokenize.word_tokenize(x))

In [94]:
Poems[1]

['He',
 'sees',
 'birds',
 'overhead',
 ',',
 'eggshells',
 'seared',
 'to',
 'their',
 'wire-wrapped',
 'claws',
 '.',
 'Their',
 'flock',
 '’',
 's',
 'light',
 'dims',
 'when',
 'cedar',
 'smoke',
 'sniffs',
 'hunger',
 'on',
 'text',
 'messages',
 'flailing',
 'in',
 'the',
 'sage',
 'brush',
 '.',
 'He',
 'unbraids',
 'rain',
 'from',
 'his',
 'hair',
 ',',
 'notices',
 ':',
 'each',
 'strand',
 'slows',
 'to',
 'a',
 'screech',
 'when',
 'glimpsed',
 'by',
 'a',
 'steer',
 'ducking',
 'the',
 'lasso',
 '’',
 's',
 'widening',
 'maw',
 '.',
 'With',
 'flames',
 'belted',
 'to',
 'his',
 'waist',
 ',',
 'he',
 'pins',
 'moth',
 'wings',
 'to',
 'each',
 'word',
 'cresting',
 '.',
 'Eyes',
 'shell-white',
 ',',
 'he',
 'ignites',
 'his',
 'gas-soaked',
 'shoes—',
 'emergency',
 'lights',
 'flare',
 'in',
 'every',
 'limb',
 'of',
 'his',
 'waking',
 '.']

In [95]:
def BiGram(dic, poem):
    a = poem
    b = poem[1:]
    for i in range(len(b)):
        if(a[i] in dic):
            if(b[i] in dic[a[i]]):
                dic[a[i]][b[i]] = dic[a[i]][b[i]]+1
            else:
                dic[a[i]][b[i]] = 1
        else:
            dic[a[i]]={b[i]:1}
            
    return(dic)

In [96]:
dicBi = {} # The dictionary for the BiGram model
for poem in Poems: #Feed it all the poems
    dicBi = BiGram(dicBi, poem) #Build the model!

In [97]:
def Next_word(word, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)

In [98]:
prev_word = "how"
generate_str = prev_word
for i in range(20):
    next_word = Next_word(prev_word, dicBi)
    prev_word = next_word
    generate_str = generate_str+" "+ next_word

In [99]:
generate_str

'how to the world , and the world , and the world , and the world , and the world ,'

In [100]:
def NGram(dictionary, poem, nGram):
    Number_of_Ngrams = len(poem)-nGram+1

    for position in range(Number_of_Ngrams):
        words = [] 
        for nWord in range(nGram):
            words = words + [poem[nWord+position]]
        temp_dic = dictionary

        for nWord in range(nGram):
            current_word = words[nWord]
            last_word = nWord+1==nGram
            if(current_word in temp_dic):
                if(last_word):
                    temp_dic[current_word] = temp_dic[current_word]+1 #Increase the Ngram Count by 1
                else:    
                    temp_dic = temp_dic[current_word]
            else:
                create_dic = 0
                if(last_word):
                    create_dic = 1
                else:
                    create_dic = {words[-1]:1}
                
                for k in range(nGram-2,nWord,-1):
                    create_dic = {words[k]:create_dic}
                temp_dic[current_word] = create_dic
                break
    return(dictionary)

In [104]:
dic4 = {}
for i in Poems:
    dic4 = NGram(dic4, i, 4)

In [105]:
def Next_word3(word1,word2,word3, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word1][word2][word3].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)
#Next_word3("NEWLINE","love","is",dic4)

In [108]:
prev_word1 = "light"
prev_word2 = "dims"#"Next_word(prev_word1, dic)"
prev_word3 = "when"# Next_word2(prev_word1,prev_word2, dic2)
generate_str = prev_word1+" "+prev_word2 + " " +prev_word3
for i in range(50):
    next_word = Next_word3(prev_word1, prev_word2,prev_word3, dic4)
    prev_word1 = prev_word2
    prev_word2 = prev_word3
    prev_word3 = next_word
    generate_str = generate_str+" "+ next_word
print(generate_str.replace("NEWLINE ","\n"))

light dims when cedar smoke sniffs hunger on text messages flailing in the sage brush . He unbraids rain from his hair , and her husband , their daughter had not been there Since the thirteenth century . Details frighten me . As the crow You undergo a beautiful catharsis trapped one night


In [ ]:
# how should i go about generating new lines

In [ ]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)